In [46]:
import pandas as pd
data_path = '../Data'

AF = f'{data_path}/FinalFeatures_AF.xlsx'
DIC = f'{data_path}/FinalFeatures_DIC.xlsx'

df1 = pd.read_excel(AF, engine='openpyxl')
df2 = pd.read_excel(DIC, engine='openpyxl')

In [68]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Class                 98 non-null     object 
 1   Cell Area             98 non-null     int64  
 2   Cell Aspect Ratio     98 non-null     float64
 3   Cell Diameter         98 non-null     float64
 4   Nucleas Area          98 non-null     int64  
 5   Nucleas Aspect Ratio  98 non-null     float64
 6   Nulcleas Diameter     98 non-null     float64
 7   Cytoplasm Area        98 non-null     int64  
 8   NC Ratio              98 non-null     float64
 9   Cellular Perimeter    98 non-null     float64
 10  Nuclear Perimeter     98 non-null     float64
 11  Cytoplasm Perimeter   98 non-null     float64
 12  MeanGray              98 non-null     float64
 13  Median                98 non-null     int64  
 14  Variance              98 non-null     float64
 15  StandaredDeviation    98 

In [69]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Class                 105 non-null    object 
 1   Cell Area             99 non-null     float64
 2   Cell Aspect Ratio     99 non-null     float64
 3   Cell Diameter         99 non-null     float64
 4   Nucleas Area          100 non-null    float64
 5   Nucleas Aspect Ratio  100 non-null    float64
 6   Nulcleas Diameter     100 non-null    float64
 7   Cytoplasm Area        95 non-null     float64
 8   NC Ratio              95 non-null     float64
 9   Cellular Perimeter    99 non-null     float64
 10  Nuclear Perimeter     100 non-null    float64
 11  Cytoplasm Perimeter   95 non-null     float64
 12  MaxGray               105 non-null    int64  
 13  MinGray               105 non-null    int64  
 14  MeanGray              105 non-null    float64
 15  Median                1

In [66]:
print(len(df1.columns), len(df2.columns))

A = set(df1.columns).intersection(set(df2.columns))
B = [i for i in df2.columns if i not in A]
print(len([i for i in df1.columns if i not in A]), len(B))

25 35
0 10


In [61]:
B

['MaxGray',
 'MinGray',
 'DifferenceVariance',
 'DifferenceEntropy',
 'InverseDifference',
 'MaxProbability',
 'SumAverage',
 'SumEntropy',
 'SumSquaresVariance',
 'SumVariance']

In [51]:
Morphological_features_SIZE = ['Cell Area',
 'Nucleas Area',
 'Cytoplasm Area',
 'Cell Diameter',
 'Cell Diameter',
 'Nulcleas Diameter',
 'Nulcleas Diameter',
 'Cell Aspect Ratio',
 'Nucleas Aspect Ratio',
 'Cell Diameter',
 'Nulcleas Diameter',
 'NC Ratio']

Morphological_features_SHAPE = ['Cellular Perimeter',
 'Nuclear Perimeter',
 'Cytoplasm Perimeter']


haralick_texture_features = [
    'Contrast',              # Contrast (local changes in image)
    'Energy',                # Energy (uniformity of image)
    'Entropy',               # Entropy (randomness of intensity)
    'Homogenity',            # Homogeneity (similarity of pixels)
    'Correlation',           # Correlation (pixel neighborhood correlation)
    'Variance',              # Variance (uncertainty degree)
    'DifferenceEntropy',     # Difference Entropy (difference among pixel entropy)
    'SumVariance',           # Sum Variance (pixel uniformity)
    'SumAverage',            # Sum Average (average of gray level)
    # 'InverseDifferenceMoment', # Not explicitly in the dataset
    'ClusterProminence',     # Cluster Prominence (normalized gray-tone spatial dependence)
    # 'ClusterTendency',         # Not explicitly in the dataset
    'ClusterShade',          # Cluster Shade (cubic normalized gray-tone)
    # 'Inertia',                # Not explicitly in the dataset
    'SumSquaresVariance',    # Sum of Square Variance (squared deviation)
    # 'AngularSecondMoment',     # Not explicitly in the dataset
    # 'InformationMeasureCorrelationI', # Not explicitly in the dataset
    # 'InformationMeasureCorrelationII' # Not explicitly in the dataset
]



In [62]:
G = Morphological_features_SHAPE + Morphological_features_SIZE + haralick_texture_features
len(G)

27